In [1]:
# %% codecell
import numpy as np
import Plot as plot
import math
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
import time
import datetime
import pickle
import os
import scipy.spatial
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib as plt
import logging
from scipy import signal
import learningAlgs as classImportLA
import dataManipulation as dataMan
from itertools import permutations
import importlib
from datetime import timedelta
from multiprocessing import Pool
import multiprocessing
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans
import timeIntervalPlotter as intervalPlotter
import pysal
import warnings
import lumping_traditional as oldLumping
import boundaryFull_SS_WeightedLumping as WLumping
from importlib import reload
from scipy.stats import rayleigh
import dataManipulation as dataMan
import matplotlib.pyplot as plt
import processData as processData
import tensorflow as tf
warnings.filterwarnings('always')
# <codecell>

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/home/sepehr/.local/lib/python3.6/site-packages/tensorflow_core/python/keras/backend.py:5747: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/home/sepehr/.keras/keras.json' mode='r' encoding='UTF-8'>



In [3]:
#*******************************************************************************
# %%codecell
reload(classImportLA)
dataFrame = processData.processingData()
#address is: /home/netlab/Desktop/thesis/data/1node1-3-5/
#/home/netlab/Desktop/thesis/data/500f80271400/
data = dataFrame.copy() #copying the dataFrame to have a copy of not edited data

print("hello")
# <codecell>

#*******************************************************************************
# %% codecell
data = processData.dataFrameManipulation(data)
numberOfStates = 255
reload(processData)
cuTrans = processData.markovianTransitionMatrixDegree1(data, numberOfStates, "CU")
normalizedCuTrans = processData.normalizingTransMatrix(cuTrans)
cuTrans_cpy = normalizedCuTrans.copy()

arrExtra = [0]
print("done")
def particling(x):
    if (x["col1"] - 1 not in data["col1"]) or (x["col1"] - 1 < 0):
        arrExtra[0] = x["time"]
        return 0

    elif x["timeIndex"] != data["timeIndex"][x["col1"] - 1]:
        arrExtra[0] = x["time"]
        return 0

    else:
        return ((x["time"] - arrExtra[0]).seconds)/6

data["periodParticle"] = data.apply(lambda x: particling(x), axis = 1) #numbering each 6 seconds in the dataFrame

print(cuTrans_cpy[0])
# <codecell>
#*******************************************************************************

def bandwidthPercentage(vectorMatrix):
    percentageIncreament = (100 / vectorMatrix.shape[0])
    percentageMatrix = []
    maxPercentage = 0
    for j in range(vectorMatrix.shape[0]):
        maxPercentage += percentageIncreament
        percentageMatrix.append([[j], maxPercentage, False])

    return percentageMatrix

def reduceMatrixAllInOne(transitionMatrix):
    reload(oldLumping)
    percentageMatrix_list = bandwidthPercentage(transitionMatrix)
    zero_cols_rows = []
    #*************removing zeros from columns and rows (matrix reduction) ****************
    for i in range(len(transitionMatrix)):
        if (np.sum(transitionMatrix[i]) == 0) and (np.sum(transitionMatrix[:,i]) == 0):
            zero_cols_rows.append(i)


    irreducible_matrix = transitionMatrix.copy()
    for i in range(len(zero_cols_rows) - 1, -1, -1):
        irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 0)
        irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 1)
        percentageMatrix_list[zero_cols_rows[i]][2] = True


    for i in range(len(percentageMatrix_list) - 1, 0, -1):
        if (percentageMatrix_list[i][2] == True) and (percentageMatrix_list[i - 1][2] == True):
            percentageMatrix_list[i - 1][0].extend(percentageMatrix_list[i][0])
            percentageMatrix_list[i - 1][1] = percentageMatrix_list[i][1]
            del percentageMatrix_list[i]

    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)



def preparingMatrixForLumping(transitionMatrix):
    reload(oldLumping)
    percentageMatrix_list = bandwidthPercentage(transitionMatrix)
    zero_cols_rows = []
    #*************removing zeros from columns and rows (matrix reduction) ****************
    for i in range(len(transitionMatrix)):
        if (np.sum(transitionMatrix[i]) == 0) and (np.sum(transitionMatrix[:,i]) == 0):
            zero_cols_rows.append(i)


    irreducible_matrix = transitionMatrix.copy()
    for i in range(len(zero_cols_rows) - 1, -1, -1):
        irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 0)
        irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 1)
        percentageMatrix_list[zero_cols_rows[i]][2] = True


    for i in range(len(percentageMatrix_list) - 1, 0, -1):
        if (percentageMatrix_list[i][2] == True) and (percentageMatrix_list[i - 1][2] == True):
            percentageMatrix_list[i - 1][0].extend(percentageMatrix_list[i][0])
            percentageMatrix_list[i - 1][1] = percentageMatrix_list[i][1]
            del percentageMatrix_list[i]

    for i in range(len(irreducible_matrix)):
        if np.sum(irreducible_matrix[i], dtype = np.float32) != 1.0:
            print(np.sum(irreducible_matrix[i], dtype = np.float32))

    return percentageMatrix_list, irreducible_matrix

def lumpingStatesOneByOne(percentageMatrix_list, irreducible_matrix):

    # savedResult = 0
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, True)

    min_degree, min_error, irreducible_matrix, best_sectors = result[0], result[1], result[2], result[3]

    zero_cols_rows = []
    for i in range(len(irreducible_matrix)):
        if (np.sum(irreducible_matrix[i]) == 0) and (np.sum(irreducible_matrix[:,i]) == 0):
            zero_cols_rows.append(i)

    if len(zero_cols_rows) != 0:
        for i in range(len(zero_cols_rows) - 1, -1, -1):
            irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 0)
            irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 1)

        percentageMatrix_list = reduceMatrix(percentageMatrix_list, zero_cols_rows)

    print("length of best lumped is: ", len(irreducible_matrix))

    percentageMatrix_list = mergeStates(best_sectors, percentageMatrix_list)

    for i in range(len(irreducible_matrix)):
        if (np.sum(irreducible_matrix[i], dtype = np.float32) != 1.0):

            print(i, np.sum(irreducible_matrix[i], dtype = np.float32))

    for i in range(irreducible_matrix.shape[0]):
        sum = 0
        sum = np.sum(irreducible_matrix[i])
        if sum != 0:
            irreducible_matrix[i] = irreducible_matrix[i]/sum

    for i in range(len(irreducible_matrix)):
        if (np.sum(irreducible_matrix[i], dtype = np.float32) != 1.0):

            print(i, np.sum(irreducible_matrix[i], dtype = np.float32))

    count = 0
    for i in range(len(percentageMatrix_list)):
        if percentageMatrix_list[i][2] == False:
            count += 1
    print("percentageMatrix length is: ", count)

    return(percentageMatrix_list, irreducible_matrix)

    # if len(irreducible_matrix) == 26:
        # break
    # result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    # return result

def reduceMatrix(percentageMatrix, zero_cols_rows):
    index_in_percentage_matrix = np.inf
    counter = 0
    for x in range(len(zero_cols_rows)):
        for i in range(len(percentageMatrix)):
            if percentageMatrix[i][2] == False:
                if zero_cols_rows[x] == counter:
                    index_in_percentage_matrix = i
                    break
                counter += 1
        percentageMatrix[index_in_percentage_matrix][2] = True
        if index_in_percentage_matrix == (len(percentageMatrix) - 1):
            if percentageMatrix[index_in_percentage_matrix - 1][2] == True:
                percentageMatrix[index_in_percentage_matrix - 1][0].extend(
                    percentageMatrix[index_in_percentage_matrix][0])

                percentageMatrix[index_in_percentage_matrix - 1][1] = percentageMatrix[index_in_percentage_matrix][1]
                del percentageMatrix[index_in_percentage_matrix]

        elif index_in_percentage_matrix == 0:
            if percentageMatrix[index_in_percentage_matrix + 1][2] == True:
                percentageMatrix[index_in_percentage_matrix][0].extend(
                    percentageMatrix[index_in_percentage_matrix + 1][0])

                percentageMatrix[index_in_percentage_matrix][1] = percentageMatrix[index_in_percentage_matrix + 1][1]
                del percentageMatrix[index_in_percentage_matrix + 1]

        else:
            if (percentageMatrix[index_in_percentage_matrix - 1][2] == True) and (
                percentageMatrix[index_in_percentage_matrix + 1][2] == True):
                leftside_bandwidth_val = 0
                rightside_bandwidth_val = 0
                if (index_in_percentage_matrix - 1) != 0:
                    leftside_bandwidth_val = percentageMatrix[index_in_percentage_matrix][1] - percentageMatrix[index_in_percentage_matrix - 2][1]
                else:
                    leftside_bandwidth_val = percentageMatrix[index_in_percentage_matrix][1]
                rightside_bandwidth_val = percentageMatrix[index_in_percentage_matrix + 1][1] - percentageMatrix[index_in_percentage_matrix - 1][1]

                if leftside_bandwidth_val <= rightside_bandwidth_val:
                    percentageMatrix[index_in_percentage_matrix - 1][0].extend(
                        percentageMatrix[index_in_percentage_matrix][0])

                    percentageMatrix[index_in_percentage_matrix - 1][1] = percentageMatrix[index_in_percentage_matrix][1]
                    del percentageMatrix[index_in_percentage_matrix]

                else:
                    percentageMatrix[index_in_percentage_matrix][0].extend(
                        percentageMatrix[index_in_percentage_matrix + 1][0])

                    percentageMatrix[index_in_percentage_matrix][1] = percentageMatrix[index_in_percentage_matrix + 1][1]
                    del percentageMatrix[index_in_percentage_matrix + 1]

            elif (percentageMatrix[index_in_percentage_matrix - 1][2] == True) and (
                percentageMatrix[index_in_percentage_matrix + 1][2] == False):

                percentageMatrix[index_in_percentage_matrix - 1][0].extend(
                    percentageMatrix[index_in_percentage_matrix][0])

                percentageMatrix[index_in_percentage_matrix - 1][1] = percentageMatrix[index_in_percentage_matrix][1]
                del percentageMatrix[index_in_percentage_matrix]

            elif (percentageMatrix[index_in_percentage_matrix - 1][2] == False) and (
                percentageMatrix[index_in_percentage_matrix + 1][2] == True):

                percentageMatrix[index_in_percentage_matrix][0].extend(
                    percentageMatrix[index_in_percentage_matrix + 1][0])

                percentageMatrix[index_in_percentage_matrix][1] = percentageMatrix[index_in_percentage_matrix + 1][1]
                del percentageMatrix[index_in_percentage_matrix + 1]

    return percentageMatrix




def mergeStates(best_sectors, percentageMatrix):
    False_state_in_percentageMatrix = np.inf
    for i in range(1, len(best_sectors)):
        if best_sectors[i] - best_sectors[i - 1] == 2:
            False_state_in_percentageMatrix = best_sectors[i]
            #it means states number (whole_subset[i] - 1) and (whole_subset[i] - 2) should merge
            #it is also the (whole_subset[i]) False and (whole_subset[i] - 1) False in percentageMatrix
            break

    index1_in_percentage_matrix = np.inf
    index2_in_percentage_matrix = np.inf
    counter = 0
    for i in range(len(percentageMatrix)):
        if percentageMatrix[i][2] == False:
            counter += 1
            if counter == (False_state_in_percentageMatrix - 1):
                index1_in_percentage_matrix = i
            elif counter == False_state_in_percentageMatrix:
                index2_in_percentage_matrix = i
                break

    # print("indices are: ", index1_in_percentage_matrix, index2_in_percentage_matrix)
    # print("values of indices: ", percentageMatrix[index1_in_percentage_matrix])
    # print("values of indices: ", percentageMatrix[index2_in_percentage_matrix])
    if (index2_in_percentage_matrix) == (index1_in_percentage_matrix + 1):
        percentageMatrix[index1_in_percentage_matrix][0].extend(
            percentageMatrix[index2_in_percentage_matrix][0])

        percentageMatrix[index1_in_percentage_matrix][1] = percentageMatrix[
            index2_in_percentage_matrix][1]

        del percentageMatrix[index2_in_percentage_matrix]

    elif (index2_in_percentage_matrix) == (index1_in_percentage_matrix + 2):
        percentageMatrix[index1_in_percentage_matrix + 1][0].extend(
            percentageMatrix[index2_in_percentage_matrix][0])

        percentageMatrix[index1_in_percentage_matrix][0].extend(
            percentageMatrix[index1_in_percentage_matrix + 1][0])

        percentageMatrix[index1_in_percentage_matrix][1] = percentageMatrix[
            index2_in_percentage_matrix][1]

        del percentageMatrix[index2_in_percentage_matrix]
        del percentageMatrix[index1_in_percentage_matrix + 1]

    return percentageMatrix

the address of the collected data files (not alligned files or CSV files): /home/sepehr/thesis/data/500f80271400/
['500f80271400.txt']
here
500f80271400.txt is in csvChecker

we have the csv file: pulling out data

   col1                time  CU
0     0 2018-11-13 02:01:32  45
1     1 2018-11-13 02:01:38  45
2     2 2018-11-13 02:01:44  51
3     3 2018-11-13 02:01:50  53
4     4 2018-11-13 02:01:56  45
now we have the processed data from pandas
hello
please enter how long would be the chunk minutes? 30
removing weekends from the data
done
[[0.34180791 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [49]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [50]:
for i in range(20, 25):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_result_" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_percentage_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  13048
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  13048
0.99999994
0.99999994
0.9999999
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  12838
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  13048
1.0000001
0.99999994
0.9999999
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
we have these many sectors to check:  13048


In [51]:
for i in range(25, 30):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_result_" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_percentage_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

1.0000001
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  12838
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
1.0000001
1.0000001
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  13048
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  12710
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
1.0000001
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  13048
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  13048


In [52]:
for i in range(30, 48):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_result_" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_percentage_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

1.0000001
0.99999994
1.0000001
1.0000001
0.99999994
1.0000001
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  13048
0.9999999
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  12838
0.99999994
1.0000001
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  12838
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
0.99999994
0.99999994
we have these many sectors to check:  13048
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  13048
0.99999994
0.99999994
0.9999999
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
0.99999994
1.0000001
we have these many sectors 

In [53]:
for i in range(20):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_result_" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_percentage_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

1.0000001
0.99999994
1.0000001
0.99999994
we have these many sectors to check:  6232
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  3723
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  1044
0.9999999
0.99999994
1.0000001
0.99999994
1.0000001
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  490
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  1088
0.9999999
0.9999999
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  1775
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  0
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  308
0.99999994
1.0000001
we have these many sec

In [54]:
arrayOfLumpes = []
indexesOfCandidates = [[] for i in range(48)]
for timeIndex in range(48):
    print(timeIndex)
    name = "/home/sepehr/thesis/APDataML/pickles/5normal_lumping_result_" + str(timeIndex) + ".pickle"
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
    
    if len(b) > 0:
        minDegree = np.inf        
        for i in range(len(b)):
            if b[i][0] < minDegree:
                minDegree = b[i][0]

        minError = np.inf
        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] < minError:
                minError = b[i][1]

        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] == minError:
                indexesOfCandidates[timeIndex].append(b[i])

name = "/home/sepehr/thesis/APDataML/5normalBoundaries.pickle"
with open(name, 'wb') as handle:
    pickle.dump(indexesOfCandidates, handle) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [5]:
for i in range(30, 40):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    name = "/home/sepehr/thesis/APDataML/pickles/normal_lumping_result_" + str(i) + ".pickle"
    with open(name, 'wb') as handle:
        pickle.dump(result, handle)

1.0000001
0.99999994
1.0000001
1.0000001
0.99999994
1.0000001
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  366134
0.9999999
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  337462
0.99999994
1.0000001
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  337462
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
0.99999994
0.99999994
we have these many sectors to check:  366134
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  366134
0.99999994
0.99999994
0.9999999
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
0.99999994
1.0000001
we have these many sec

In [6]:
for i in range(17, 20):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    name = "/home/sepehr/thesis/APDataML/pickles/normal_lumping_result_" + str(i) + ".pickle"
    with open(name, 'wb') as handle:
        pickle.dump(result, handle)

0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  264788
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  256785
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  279766


In [7]:
for i in range(14, 17):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    name = "/home/sepehr/thesis/APDataML/pickles/normal_lumping_result_" + str(i) + ".pickle"
    with open(name, 'wb') as handle:
        pickle.dump(result, handle)

0.99999994
0.99999994
we have these many sectors to check:  195218
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  167340
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
1.0000001
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  98428


In [12]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [17]:
for i in [6, 9, 11, 12, 44]:
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    name = "/home/sepehr/thesis/APDataML/pickles/normal_lumping_result_" + str(i) + ".pickle"
    with open(name, 'wb') as handle:
        pickle.dump(result, handle)    

0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  8800
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
we have these many sectors to check:  0
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  0
0.99999994
0.99999994
1.0000001
0.99999994
we have these many sectors to check:  1776
0.99999994
0.99999994
0.99999994
0.9999999
1.0000001
0.99999994
we have these many sectors to check:  13156


In [20]:
indexesOfCandidates = [[] for i in range(48)]
print(indexesOfCandidates)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [27]:
arrayOfLumpes = []
indexesOfCandidates = [[] for i in range(48)]
for timeIndex in range(48):
    print(timeIndex)
    name = "/home/sepehr/thesis/APDataML/normalPickles/normal_lumping_result_" + str(timeIndex) + ".pickle"
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
    
    if len(b) > 0:
        minDegree = np.inf        
        for i in range(len(b)):
            if b[i][0] < minDegree:
                minDegree = b[i][0]

        minError = np.inf
        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] < minError:
                minError = b[i][1]

        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] == minError:
                indexesOfCandidates[timeIndex].append(b[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [35]:
# indexesOfCandidates
name = "/home/sepehr/thesis/APDataML/normalBoundaries-30min.pickle"
with open(name, 'wb') as handle:
    pickle.dump(indexesOfCandidates, handle) 

In [30]:
name = "/home/sepehr/thesis/APDataML/normalBoundaries_30min.pickle"
with open(name, 'rb') as handle:
    b = pickle.load(handle)

In [34]:
b[0]

[(1.0,
  40.0799560546875,
  array([[0.83597884, 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.3968254 , 0.20052083, 0.03125   , 0.        , 0.        ,
          0.        ],
         [0.3       , 0.14285714, 0.2       , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.25      , 0.        ,
          0.        ],
         [0.33333333, 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ]]),
  [-1, 19, 30, 46, 60, 67, 79])]